<a href="https://colab.research.google.com/github/S-EGK/Contollers/blob/main/Non-Linear%20Controllers/Adaptive%20Controllers/Adaptive%20Trajectory%20Controller%20for%20a%20Pendulum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 5 Problem 14.2
## Srikanth EGK
### M13998009

## Libraries

In [ ]:
# importing libraries
from math import cos, sin
import numpy as np
from random import random
import scipy.signal as signal
import scipy.linalg as linalg
from mpl_toolkits.mplot3d import Axes3D
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.patches import Circle

## Parameters

### Simulation Parameters

In [ ]:
dt = 0.1
Tfinal = 40
Ts = dt
Tsteps = Tfinal/Ts # number of frames
frames = int(Tsteps)

### System Parameters

In [ ]:
# States
x1 = 0.01
x2 = 1
a_hat = 0

states = np.array([x1, x2, a_hat])

a = 2
gamma = 0.5
k1 = 2

### Datat Collection and Plot Parameters

In [ ]:
# Data Collectors
f_data = []
x1_data = []
x2_data = []
a_hat_data = []
a_data = []
psi_r_data = []
psi_r_dot_data = []

# Animation Parameters
rod_x = [0, sin(x1)]
rod_y = [0, -cos(x1)]

fig = plt.figure(figsize = (10,5))
ax1 = plt.subplot(1,1,1)
ax1.set_xlim((-10,10))
ax1.set_ylim((-3,3))

rod, = ax1.plot(rod_x, rod_y, 'b', lw = 2)
bob, = ax1.plot(sin(x1), -cos(x1), 'g', marker = 'o', ms = 10)

In [ ]:
def anipts(x1):
  rod_x = [0, sin(x1)]
  rod_y = [0, -cos(x1)]
  bob_x, bob_y = sin(x1), -cos(x1)

  return rod_x, rod_y, bob_x, bob_y

## Dynamics

In [ ]:
def dyna(t,y,f,ref):
  x1 = y[0]
  x2 = y[1]
  a_hat = y[2]
  psi_r_dot = ref[1]
  psi_r_ddot = ref[2]
  
  u = f
  
  x1_dot = x2
  x2_dot = -a*(x2 + psi_r_dot)*np.absolute((x2 + psi_r_dot)) - psi_r_ddot + u
  a_hat_dot = gamma*(x2 + k1*x1)*(x2 + psi_r_dot)*np.absolute((x2 + psi_r_dot))

  return x1_dot, x2_dot, a_hat_dot

## Reference Trajectory

In [ ]:
def ref_traj(i):
  w = np.pi/20
  psi_r = sin(w*i)
  psi_r_dot = w*cos(w*i)
  psi_r_ddot = w*w*sin(w*i)

  ref = np.array([psi_r, psi_r_dot, psi_r_ddot])

  return ref

## Adaptive Controller

In [ ]:
def adap_ctrl(y, ref):
  x1 = y[0]
  x2 = y[1]
  a_hat = y[2]
  psi_r_dot = ref[1]
  psi_r_ddot = ref[2]

  u = a_hat*(x2 + psi_r_dot)*np.absolute((x2 + psi_r_dot)) - k1*x2 - x1 + psi_r_ddot - 0.5*(x2+k1*x1)
  return u

## Solver

In [ ]:
def step(states, dt,i):
  x1 = states[0]
  x2 = states[1]

  ref = ref_traj(i)

  f = adap_ctrl(states, ref)
  f_data.append(f)

  sol = solve_ivp(lambda t,y: dyna(t,y,f,ref), [0,dt], [x1, x2, a_hat], t_eval = np.linspace(0,dt,100))
  states = sol.y[:,-1].T

  return states, ref

## Iterating

In [ ]:
def drawframe(i):
  global states
  states, ref = step(states, dt,i)
  x1 = states[0]
  x2 = states[1]
  a_hat = states[2]
  psi_r = ref[0]
  psi_r_dot = ref[1]

  x1_data.append(states[0]+ref[0])
  x2_data.append(states[1]+ref[1])
  a_hat_data.append(states[2])
  psi_r_data.append(ref[0])
  psi_r_dot_data.append(ref[1])
  a_data.append(a)

  rod_x, rod_y, bob_x, bob_y = anipts(states[0]+ref[0])

  rod.set_data(rod_x, rod_y)
  bob.set_data(bob_x, bob_y)

  return rod, bob

## Animation

In [ ]:
from matplotlib import animation
# blit=True re-draws only the parts that have changed.
anim = animation.FuncAnimation(fig, drawframe, frames=frames, interval=40, blit=True)

In [ ]:
from IPython.display import HTML
HTML(anim.to_html5_video())

## Plots

In [ ]:
fig1 = plt.figure(figsize=(17,22), dpi=90)

ax1 = fig1.add_subplot(4, 3, 1)
ax1.set_xlabel('Time Step')
ax1.set_ylabel('x1')
ax1.set_title('State Plot of x1')
x11 = ax1.plot(range(len(x1_data)),x1_data, label='x1')
r1 = ax1.plot(range(len(psi_r_data)),psi_r_data, 'r:', label='x1_ref')
ax1.legend(loc = "best")

ax2 = fig1.add_subplot(4, 3, 2)
ax2.set_xlabel('Time Step')
ax2.set_ylabel('x2')
ax2.set_title('State Plot of x2')
x21 = ax2.plot(range(len(x2_data)),x2_data, label='x2')
r21 = ax2.plot(range(len(psi_r_dot_data)),psi_r_dot_data, 'r:', label='x2_ref')
ax2.legend(loc = "best")

ax3 = fig1.add_subplot(4, 3, 3)
ax3.set_xlabel('x1')
ax3.set_ylabel('x2')
ax3.set_title('x1 - x2')
x1x21 = ax3.plot(x1_data,x2_data, label='x1-x2')
rr1 = ax3.plot(psi_r_data,psi_r_dot_data, 'r:', label='x1_ref - x2_ref')
ax3.legend(loc = "best")

ax4 = fig1.add_subplot(4, 3, 4)
ax4.set_xlabel('Time Step')
ax4.set_ylabel('f')
ax4.set_title('f vs time step')
f1 = ax4.plot(range(len(f_data)),f_data, label='input')
ax4.legend(loc = "best")

ax5 = fig1.add_subplot(4, 3, 5)
ax5.set_xlabel('Time Step')
ax5.set_ylabel('a')
ax5.set_title('a vs time step')
a1 = ax5.plot(range(len(a_data)),a_data, label='a')
aa1 = ax5.plot(range(len(a_hat_data)),a_hat_data, 'r:', label='a_hat')
ax5.legend(loc = "best")